In [55]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
from collections import Counter

# Charger le dataset non prétraité (brut) et prétraité
df_raw = pd.read_csv('../datasets/all_clean_.csv')  # Texte brut

# Charger les données finales dans la variable df
df = pd.read_csv('../datasets/preprocessed_df.csv')
# Étape 2 : Nettoyer les données
df = df.drop(columns=['is_eng'])  # Suppression des colonnes inutiles
df = df.dropna(subset=['text'])  # Supprimer les lignes avec des valeurs manquantes

# Étape 3 : Préparer les features (X) et les labels (y)
text_data = df['text']  # Texte brut
numeric_data = df[['len', 'len_no_spaces', 'n_words', 'n_special_chars', 'n_capital_letters', 'entities','w_free','w_click','w_claim', 'w_offer','w_win', 'w_prize', 'w_want']]  # Colonnes numériques
y = df['label']  # Labels (spam/ham)

# Étape 4 : Transformer le texte en vecteurs TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_text = vectorizer.fit_transform(text_data)

# Étape 5 : Combiner les données TF-IDF avec les colonnes numériques
X_combined = np.hstack((X_text.toarray(), numeric_data.to_numpy()))

# Étape 6 : Séparer les données en entraînement/test avant d'appliquer SMOTE
X_train, X_test, y_train, y_test, text_train, text_test = train_test_split(
    X_combined, y, text_data, test_size=0.1, random_state=41                                                                                                                                                        , #random_state=41
)

# Étape 7 : Appliquer SMOTE uniquement sur les données d'entraînement
smote = SMOTE(random_state=41)
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Vérification des tailles après SMOTE
print("Taille après SMOTE :")
print("X_train_resampled :", X_train_resampled.shape, "y_train_resampled :", len(y_train_resampled))

Taille après SMOTE :
X_train_resampled : (8132, 5013) y_train_resampled : 8132


In [56]:
# # Importer les bibliothèques nécessaires
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
# import numpy as np
# from gensim.models import Word2Vec
# df_raw = pd.read_csv('../datasets/all_clean_.csv')
# # Charger le dataset
# df = pd.read_csv('../datasets/preprocessed_df.csv')

# # Étape 3 : Préparer les features (X) et les labels (y)
# text_data = df['text']  # Texte brut
# numeric_data = df[['len', 'len_no_spaces', 'n_words', 'n_special_chars', 'n_capital_letters', 'emails', 'urls', 'phones']]  # Colonnes numériques
# y = df['label']  # Labels (spam/ham)


# # Étape 5 : Entraîner un modèle Word2Vec sur les textes
# model_w2v = Word2Vec(sentences=text_data, vector_size=100, window=5, min_count=1, workers=4)

# # Étape 6 : Fonction pour obtenir les embeddings Word2Vec pour un document
# def get_word2vec_vector(doc, model_w2v):
#     # Initialiser un vecteur vide pour le document
#     doc_vector = np.zeros(model_w2v.vector_size)
    
#     # Ajouter les vecteurs des mots dans le document
#     word_count = 0
#     for word in doc:
#         if word in model_w2v.wv:
#             doc_vector += model_w2v.wv[word]
#             word_count += 1
    
#     # Si aucun mot n'a été trouvé dans le modèle, retourner un vecteur zéro
#     if word_count > 0:
#         doc_vector /= word_count  # Moyenne des vecteurs
#     return doc_vector

# # Étape 7 : Appliquer la fonction de transformation à chaque document
# X_text_w2v = np.array([get_word2vec_vector(doc, model_w2v) for doc in text_data])

# # Étape 8 : Combiner les embeddings Word2Vec avec les colonnes numériques
# X_combined = np.hstack((X_text_w2v, numeric_data.to_numpy()))

# # Étape 9 : Séparer les données en entraînement/test avant d'appliquer SMOTE
# X_train, X_test, y_train, y_test = train_test_split(
#     X_combined, y, test_size=0.3, random_state=41
# )

# # Étape 10 : Appliquer SMOTE uniquement sur les données d'entraînement
# smote = SMOTE(random_state=41)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# # Vérification des tailles après SMOTE
# print("Taille après SMOTE :")
# print("X_train_resampled :", X_train_resampled.shape, "y_train_resampled :", len(y_train_resampled))

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# utiliser le modèle random forest pour prédire les données
from sklearn.ensemble import RandomForestClassifier
#utiliser le modele de reseau de neurones pour prédire les données
from sklearn.neural_network import MLPClassifier
#utliser le model bert pour prédire les données


# Entraîner le modèle
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Prédictions et évaluation
y_pred = clf.predict(X_test)
print("Précision Logistic Regression :", accuracy_score(y_test, y_pred))

Précision Logistic Regression : 0.9749582637729549


In [58]:
from sklearn.metrics import classification_report, confusion_matrix

print("Matrice de confusion :")
print(confusion_matrix(y_test, y_pred))

print("Rapport de classification :")
print(classification_report(y_test, y_pred))


Matrice de confusion :
[[450   0]
 [ 15 134]]
Rapport de classification :
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       450
           1       1.00      0.90      0.95       149

    accuracy                           0.97       599
   macro avg       0.98      0.95      0.97       599
weighted avg       0.98      0.97      0.97       599



In [59]:
# Identifier les indices des faux positifs et négatifs
faux_positifs = np.where((y_pred == 1) & (y_test == 0))[0]  # FP : Prédit spam mais vrai ham
faux_negatifs = np.where((y_pred == 0) & (y_test == 1))[0]  # FN : Prédit ham mais vrai spam

# Étape 5 : Visualiser les textes bruts pour les faux positifs et négatifs
print("Faux positifs (hams classés comme spams) :")
for index in faux_positifs:
    print(f"Texte brut : {df_raw['text'].iloc[index]}")  # Affiche le texte brut
    print(f"Prédiction : {y_pred[index]}, Vrai label : {y_test.iloc[index]}")
    print()

Faux positifs (hams classés comme spams) :


In [60]:
print("Faux négatifs (spams classés comme hams) :")
for index in faux_negatifs:
    print(f"Texte brut : {df_raw['text'].iloc[index]}")  # Affiche le texte brut
    print(f"Prédiction : {y_pred[index]}, Vrai label : {y_test.iloc[index]}")
    print()

Faux négatifs (spams classés comme hams) :
Texte brut : n a d a [entity]
Prédiction : 0, Vrai label : 1

Texte brut : want to be dropped from our list do not reply to this email copy and paste this link into your browser bisops com rmm htm computer technologies 848 n rainbow blvd 316 las vegas nv[entity]
Prédiction : 0, Vrai label : 1

Texte brut : you gape for [explicit]like you had seen in [explicit] films now you have chance to do it become stronger show your volume worried it won t work don t be afraid delete me [entity]
Prédiction : 0, Vrai label : 1

Texte brut : alpha male plus the only multiple [explicit] supplement for men prevent premature ejaculaton become the ultimate [explicit] machine multiple [explicit]with no erection loss your easy to use solution is here [entity]
Prédiction : 0, Vrai label : 1

Texte brut : goodbye gilt chimiquepretend berra galacticablaze northeast whombloodstream hardhat entertainhaley periodic gyroscope arachnidsubstantive hookup formic pliocenecha